In [34]:
import urllib
import requests
import itertools
from copy import deepcopy as mcopy
from music21 import *
import threading
import time

from ksxml import permute, \
    nth_permutation_indices, \
    send_m21_object, \
    write_midi_file, \
    send_transport_command, \
    stream_generator_from_chord_arrays

In [2]:
send_transport_command('a', 'play')

<Response [200]>

In [3]:
# source material
n1 = ['C3', 'D3', 'G3', 'A3']
n2 = ['C3', 'e3', 'G3', 'b3'] 
n3 = ['D3', 'E3', 'A3', 'B3']

chords_with_names = [n1, n1, n2, n2, n3, n3]

In [30]:
stream_gen = stream_generator_from_chord_arrays(chords_with_names, 0, 400)
streams = [s for s in stream_gen]
full_stream = stream.Stream()
for s in streams:
    s.insert(clef.BassClef())
    full_stream.append(s)
    
write_midi_file(full_stream.flat, '/session/default.mid')

In [64]:
def stop():
    for thread in threading.enumerate():
        if thread.isAlive() and thread.name == 'ks-playback':
            thread.cancel()
    send_transport_command('a', 'stop')

In [65]:
def send_stream_by_index(streams, index, end = 100):
    t = threading.Timer(2.1, send_stream_by_index, [streams, index + 1, end])
    t.name = 'ks-playback'
    t.daemon = True
    t.start()
    send_m21_object('a', streams[index])
    if index > end:
        t.cancel()

In [66]:
def start(streams, end=10):
    stop()
    send_stream_by_index(streams, 0, end)
    time.sleep(0.05)
    send_transport_command('a', 'reload')
    send_transport_command('a', 'play')

In [67]:
start(streams, 100)

In [14]:
# Test pushing score to browser client from a m21 stream object
send_m21_object('a', streams[3])

<Response [200]>

In [11]:
# Test writing a score file
write_midi_file(streams[0], '/session/default.mid')

In [33]:
# Test Transport Controls
send_transport_command('a', 'play')

<Response [200]>

In [136]:
send_transport_command('a', 'stop')

<Response [200]>

In [40]:
send_transport_command('a', 'reload')

<Response [200]>

In [2]:
from ksxml import strip_ids_from_xmlstr